In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [15]:
##Saving the baseline model
import json
import torch
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch.optim import AdamW
from torch.utils.data import DataLoader
from peft import get_peft_model, LoraConfig, TaskType
from tqdm import tqdm
# Setup
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load Data
with open("/kaggle/input/spiderdatasetnlp/spider_data/train_spider.json", "r") as f:
    train_data = json.load(f)

with open("/kaggle/input/spiderdatasetnlp/spider_data/tables.json", "r") as f:
    tables_data = json.load(f)


db_schemas = {}
for db in tables_data:
    db_id = db["db_id"]
    table_names = db["table_names_original"]
    column_names = db["column_names_original"]

    table_to_columns = {table: [] for table in table_names}
    for table_idx, column_name in column_names:
        if table_idx == -1:
            continue
        table = table_names[table_idx]
        table_to_columns[table].append(column_name)

    db_schemas[db_id] = table_to_columns

#train_data = train_data[:500]


input_texts = []
output_texts = []

for item in train_data:
    db_id = item['db_id']
    question = item['question']
    sql = item['query']
    db_schema = db_schemas[db_id]


    schema_str = "\n".join(f"- {t}: {', '.join(cols)}" for t, cols in db_schema.items())

    input_text = f"""


    Database Schema:
    {schema_str}
    
    Question:
    {question}
    
    Provide SQL Query: 
    """


    output_text = sql

    input_texts.append(input_text)
    output_texts.append(output_text)

    # print(f"OUTPUT : {output_text}")

dataset = Dataset.from_dict({
    "input_text": input_texts,
    "output_text": output_texts
})

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")

def tokenize_function(examples):
    model_inputs = tokenizer(examples["input_text"], max_length=512, padding="max_length", truncation=True)
    labels = tokenizer(examples["output_text"], max_length=128, padding="max_length", truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["input_text", "output_text"])

train_dataloader = DataLoader(
    tokenized_dataset,
    batch_size=4,
    shuffle=True,
    collate_fn=lambda x: {
        key: torch.tensor([d[key] for d in x]) for key in x[0]
    }
)
# Model
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")
peft_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=16, lora_alpha=32, lora_dropout=0.1)
model = get_peft_model(model, peft_config)
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-4)

# Training
model.train()
for epoch in range(5):
    total_loss = 0
    for batch in tqdm(train_dataloader,desc="train:"):
        optimizer.zero_grad()
        batch = {k: v.to(device) for k, v in batch.items()}
    
        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=batch["labels"]
        )
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    
        total_loss += loss.item()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1} - Average Training Loss: {avg_loss:.4f}")

# Save
model.save_pretrained("./model_output")
tokenizer.save_pretrained("./model_output")

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

train:: 100%|██████████| 1750/1750 [16:07<00:00,  1.81it/s]


Epoch 1 - Average Training Loss: 1.0602


train:: 100%|██████████| 1750/1750 [16:05<00:00,  1.81it/s]


Epoch 2 - Average Training Loss: 0.1780


...

train:: 100%|██████████| 1750/1750 [16:06<00:00,  1.81it/s]


Epoch 4 - Average Training Loss: 0.1213


train:: 100%|██████████| 1750/1750 [16:07<00:00,  1.81it/s]


Epoch 5 - Average Training Loss: 0.1182


('./model_output/tokenizer_config.json',
 './model_output/special_tokens_map.json',
 './model_output/spiece.model',
 './model_output/added_tokens.json')

In [16]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
from peft import PeftModel
import json

# Setup
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load LLM1 (Table Selector)
llm1_tokenizer = T5Tokenizer.from_pretrained("/kaggle/input/llm1_schemalinking/pytorch/default/1/llm1_schemalinking/model_output")
llm1_model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")
llm1_model = PeftModel.from_pretrained(llm1_model, "/kaggle/input/llm1_schemalinking/pytorch/default/1/llm1_schemalinking/model_output")
llm1_model.to(device)
llm1_model.eval()

# Load LLM2 (SQL Generator)
llm2_tokenizer = T5Tokenizer.from_pretrained("/kaggle/working/model_output")
llm2_model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")
llm2_model = PeftModel.from_pretrained(llm2_model, "/kaggle/working/model_output")
llm2_model.to(device)
llm2_model.eval()

# Load schema
with open("/kaggle/input/spiderdatasetnlp/spider_data/tables.json", "r") as f:
    tables_data = json.load(f)

db_schemas = {}
for db in tables_data:
    db_id = db["db_id"]
    table_names = db["table_names_original"]
    column_names = db["column_names_original"]

    table_to_columns = {table: [] for table in table_names}
    for table_idx, column_name in column_names:
        if table_idx == -1:
            continue
        table = table_names[table_idx]
        table_to_columns[table].append(column_name)

    db_schemas[db_id] = table_to_columns

# ---------- Pipeline Inference ---------- #

def get_relevant_tables(question, db_schema):
    """Uses LLM1 to get relevant tables."""
    schema_str = "\n".join(f"- {t}: {', '.join(cols)}" for t, cols in db_schema.items())
    input_text = f"""
Database Schema:
{schema_str}

Question:
{question}

Which tables are relevant?"""

    inputs = llm1_tokenizer(input_text, return_tensors="pt", truncation=True, padding=True).to(device)
    outputs = llm1_model.generate(**inputs, max_new_tokens=64)
    prediction = llm1_tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Parse table flags (e.g., "Tables: students -> True, courses -> False")
    table_flags = {}
    for part in prediction.split(","):
        if "->" in part:
            table, flag = part.strip().split("->")
            table_flags[table.strip()] = flag.strip().lower() == "true"
    return [table for table, is_used in table_flags.items() if is_used]

def generate_sql(question, reduced_schema):
    """Uses LLM2 to generate SQL from question and pruned schema."""
    schema_str = "\n".join(f"- {t}: {', '.join(cols)}" for t, cols in reduced_schema.items())
    input_text = f"""

Database Schema:
{schema_str}

Question:
{question}

Provide SQL Query:"""

    inputs = llm2_tokenizer(input_text, return_tensors="pt", truncation=True, padding=True).to(device)
    outputs = llm2_model.generate(**inputs, max_new_tokens=128)
    return llm2_tokenizer.decode(outputs[0], skip_special_tokens=True)

# ---------- Run Example ---------- #
def run_pipeline(question, db_id):
    full_schema = db_schemas[db_id]
    relevant_tables = get_relevant_tables(question, full_schema)

    if not relevant_tables:
        return "No relevant tables found."

    reduced_schema = {}
    for t in relevant_tables:
        t_clean = t.lower().replace("tables:", "").replace("table:", "").strip()
        if t_clean in full_schema:
            reduced_schema[t_clean] = full_schema[t_clean]

    sql = generate_sql(question, reduced_schema)
    return sql


In [18]:
import torch
import json
from tqdm import tqdm
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer
from peft import PeftModel

# Setup
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load LLM1 (table selector)
llm1_tokenizer = T5Tokenizer.from_pretrained("/kaggle/input/llm1_schemalinking/pytorch/default/1/llm1_schemalinking/model_output")
llm1_base = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")
llm1_model = PeftModel.from_pretrained(llm1_base, "/kaggle/input/llm1_schemalinking/pytorch/default/1/llm1_schemalinking/model_output")
llm1_model.to(device)
llm1_model.eval()

# Load LLM2 (SQL generator)
tokenizer = T5Tokenizer.from_pretrained("/kaggle/working/model_output")
base_model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")
model = PeftModel.from_pretrained(base_model, "/kaggle/working/model_output")
model.to(device)
model.eval()

# Load validation data
with open("/kaggle/input/spiderdatasetnlp/spider_data/dev.json", "r") as f:
    val_data = json.load(f)

#val_data = val_data[:100]

with open("/kaggle/input/spiderdatasetnlp/spider_data/tables.json", "r") as f:
    tables_data = json.load(f)

# Build schema dictionary
db_schemas = {}
for db in tables_data:
    db_id = db["db_id"]
    table_names = db["table_names_original"]
    column_names = db["column_names_original"]

    table_to_columns = {table: [] for table in table_names}
    for table_idx, column_name in column_names:
        if table_idx == -1:
            continue
        table = table_names[table_idx]
        table_to_columns[table].append(column_name)

    db_schemas[db_id] = table_to_columns

# Helper: get relevant tables using LLM1
def get_relevant_tables(question, db_schema):
    schema_str = "\n".join(f"- {t}: {', '.join(cols)}" for t, cols in db_schema.items())
    input_text = f"""
Database Schema:
{schema_str}

Question:
{question}

In case of JOIN consider all tables to be relevant

Which tables are relevant? """

    inputs = llm1_tokenizer(input_text, return_tensors="pt", truncation=True, padding=True).to(device)
    outputs = llm1_model.generate(**inputs, max_new_tokens=64)
    prediction = llm1_tokenizer.decode(outputs[0], skip_special_tokens=True)

    table_flags = {}
    for part in prediction.split(","):
        if "->" in part:
            table, flag = part.strip().split("->")
            table_flags[table.strip()] = flag.strip().lower() == "true"
    return [table for table, is_used in table_flags.items() if is_used]

# Preprocessing: Generate input_texts and output_texts using integrated pipeline
input_texts = []
output_texts = []
db_ids = []

for item in tqdm(val_data, desc="Preparing data"):
    db_id = item['db_id']
    question = item['question']
    gold_sql = item['query']
    full_schema = db_schemas[db_id]

    # LLM1: Select relevant tables
    relevant_tables = get_relevant_tables(question, full_schema)
    if not relevant_tables:
        reduced_schema = full_schema  # fallback to full
    else:
        reduced_schema = {}
        for t in relevant_tables:
            t_clean = t.lower().replace("tables:", "").replace("table:", "").strip()
            if t_clean in full_schema:
                reduced_schema[t_clean] = full_schema[t_clean]


    schema_str = "\n".join(f"- {t}: {', '.join(cols)}" for t, cols in reduced_schema.items())
    input_text = f"Database Schema:\n{schema_str}\n\nQuestion:\n{question}\n\nProvide SQL Query:"

    input_texts.append(input_text)
    output_texts.append(gold_sql)
    db_ids.append(db_id)

# Hugging Face dataset
dataset = Dataset.from_dict({
    "input_text": input_texts,
    "output_text": output_texts
})

# Tokenization
def tokenize_function(examples):
    model_inputs = tokenizer(examples["input_text"], max_length=512, padding="max_length", truncation=True)
    labels = tokenizer(examples["output_text"], max_length=128, padding="max_length", truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["input_text", "output_text"])

val_dataloader = DataLoader(
    tokenized_dataset,
    batch_size=4,
    collate_fn=lambda x: {
        key: torch.tensor([d[key] for d in x]) for key in x[0]
    }
)

# Evaluation
preds = []
labels = []

for batch in tqdm(val_dataloader, desc="Evaluating"):
    batch = {k: v.to(device) for k, v in batch.items()}

    with torch.no_grad():
        generated_ids = model.generate(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            num_beams=5,
            early_stopping=False,
            max_length=128
        )

    decoded_preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(batch["labels"], skip_special_tokens=True)

    preds.extend(decoded_preds)
    labels.extend(decoded_labels)

# Output JSON for evaluation
prediction_entries = []
for pred, gold, db_id in zip(preds, labels, db_ids):
    prediction_entries.append({
        "query": pred,
        "db_id": db_id
    })

with open("predict.json", "w") as f:
    json.dump(prediction_entries, f, indent=2)

print("✅ Saved predictions to predict.json")

Preparing data: 100%|██████████| 1034/1034 [22:19<00:00,  1.30s/it]


Map:   0%|          | 0/1034 [00:00<?, ? examples/s]

Evaluating: 100%|██████████| 259/259 [10:59<00:00,  2.55s/it]

✅ Saved predictions to predict.json


In [19]:
import json

# Load gold data from Spider dev.json
with open("/kaggle/input/spiderdatasetnlp/spider_data/dev.json") as f:
    gold_data = json.load(f)

# Write gold.sql (each line: <GOLD_SQL> \t <DB_ID>)
with open("gold.sql", "w") as f:
    for example in gold_data:
        f.write(f"{example['query']}\t{example['db_id']}\n")

# Write pred.sql (each line: predicted SQL)
# `preds` must be in the same order as `gold_data`
with open("pred.sql", "w") as f:
    for pred in preds:
        f.write(pred.strip() + "\n")


In [13]:
!git clone https://github.com/taoyds/spider.git

Cloning into 'spider'...
remote: Enumerating objects: 386, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 386 (delta 18), reused 15 (delta 13), pack-reused 356 (from 1)
Receiving objects: 100% (386/386), 44.93 MiB | 38.24 MiB/s, done.
Resolving deltas: 100% (112/112), done.


In [20]:
!python3 /kaggle/working/spider/evaluation.py \
    --gold gold.sql \
    --pred pred.sql \
    --etype match \
    --db /kaggle/input/spiderdatasetnlp/spider_data/database \
    --table /kaggle/input/spiderdatasetnlp/spider_data/tables.json

medium pred: SELECT T1.Name , T1.Country , T1.Age FROM singer AS T1 JOIN singer AS T2 ON T1.Singer_ID = T2.Singer_ID ORDER BY T2.Age DESC
medium gold: SELECT name ,  country ,  age FROM singer ORDER BY age DESC

medium pred: SELECT T2.Song_Name , T2.Song_release_year FROM singer AS T1 JOIN singer AS T2 ON T1.Singer_ID = T2.Singer_ID ORDER BY Age DESC LIMIT 1
medium gold: SELECT song_name ,  song_release_year FROM singer ORDER BY age LIMIT 1

medium pred: SELECT max(capacity) , avg(average) FROM stadium
medium gold: select max(capacity), average from stadium

eval_err_num:1
medium pred: SELECT T1.Name , COUNT(*) FROM stadium AS T1 JOIN concerts AS T2 ON T1.Stadium_ID = T2.Stadium_ID GROUP BY T1.Stadium_ID
medium gold: SELECT T2.name ,  count(*) FROM concert AS T1 JOIN stadium AS T2 ON T1.stadium_id  =  T2.stadium_id GROUP BY T1.stadium_id

eval_err_num:2
medium pred: SELECT T1.Name , COUNT(*) FROM stadium AS T1 JOIN concerts AS T2 ON T1.Stadium_ID = T2.Stadium_ID GROUP BY T1.Stadium_ID
